# Testing only large model on val data

In [1]:
# Load model credentials
import os
from dotenv import load_dotenv

load_dotenv()

# Loading large model credentials
LARGE_AUTH = os.getenv("LARGE_AUTH")
LARGE_TOKEN_KEY = os.getenv("LARGE_TOKEN_KEY")
LARGE_TOKEN_ID = os.getenv("LARGE_TOKEN_ID")

In [32]:
# Loading validation data
import json

with open(r"..\data\test\val.json", "r", encoding="utf-8") as file:
    data = json.load(file)

val_data = json.dumps(data, indent=4, ensure_ascii=False)
item = data[0]
str(item["choices"])
item

{'qid': 'val_0001',
 'question': 'Đoạn thông tin:\n[1] Tiêu đề: Khỉ thí nghiệm\nNội dung: Khỉ thí nghiệm là thuật ngữ chỉ về các loài linh trưởng (trừ con người), thông thường là các loài khỉ được sử dụng trong thí nghiệm y khoa (NHPs). Khỉ bao gồm các loài khỉ thực sự và các loài khỉ lớn (Ape) được sử dụng cho các mục đích thí nghiệm khoa học, nhất là trong y học. Xuất phát từ sự tương đồng của về cấu trúc sinh học giữa các loài khỉ và người do đó chúng thường được nuôi để làm vật thí nghiệm liên quan đến y khoa, trên cơ sở đó sẽ nhân rộng ra cho con người.\nCó 22 loài khỉ đuôi dài, trong đó một số được sử dụng thường xuyên trong các thí nghiệm khoa học. Khỉ vàng là nguồn nguyên liệu đầu để sản xuất hàng chục triệu liều văcxin bại liệt mỗi năm, góp phần vào việc thanh toán hoàn toàn bệnh bại liệt tại Việt Nam vào những năm 2000 Được sử dụng trong sản xuất vacxin chống bệnh bại liệt trẻ em, làm vật mẫu, đối tượng nghiên cứu khoa học. Khỉ vàng được lựa chọn là đối tượng nghiên cứu của c

In [30]:
# Prompt
prompt = f"""
Bạn là một trợ lý chuyên gia đa lĩnh vực (lịch sử, toán, văn học, khoa học, lập trình, v.v.).
Nhiệm vụ: sau khi nhận phần câu hỏi và các lựa chọn dưới đây, hãy **đánh giá kỹ lưỡng nội bộ** từng lựa chọn và **chọn đáp án đúng nhất**.

QUY TẮC XUẤT KẾT QUẢ (rất quan trọng — tuân thủ chính xác):
1. Suy nghĩ từng bước step-by-step để giải câu hỏi dưới đây
2. Dòng tiếp: **Giải thích súc tích** 
1. Nếu câu hỏi là bài toán cần tính toán: **ghi công thức / biểu thức chính** và **kết quả cuối cùng**


BÂY GIỜ XỬ LÝ INPUT SAU:
Câu hỏi: {item["question"]}
Các lựa chọn: {item["choices"]}
"""


In [31]:
import requests
headers = {
    "Authorization": LARGE_AUTH,
    "Token-id": LARGE_TOKEN_ID,
    "Token-key": LARGE_TOKEN_KEY, 
    "Content-Type": "application/json",
}

json_data = {
    "model": "vnptai_hackathon_large",
    "messages": [
        {
            "role": "user",
            "content": prompt,
        },
    ],
    "temperature": 0.5,
    "top_p": 1.0,
    "top_k": 20,
    "n": 1,
    "max_completion_tokens": 1024,
}

response = requests.post("https://api.idg.vnpt.vn/data-service/v1/chat/completions/vnptai-hackathon-large", headers=headers, json=json_data)
response.json()['choices'][0]['message']['content']

'Để trả lời câu hỏi này, tôi sẽ đánh giá từng lựa chọn dựa trên thông tin cung cấp:\n\n1. **Khỉ đuôi dài**: Thông tin cho biết có 22 loài khỉ đuôi dài, và một số được sử dụng thường xuyên trong các thí nghiệm khoa học. Tuy nhiên, không cụ thể rằng loài này được sử dụng phổ biến nhất trong sản xuất vắc-xin bại liệt và H5N1 ở Việt Nam.\n\n2. **Khỉ vàng**: Thông tin cụ thể rằng khỉ vàng là nguồn nguyên liệu đầu để sản xuất hàng chục triệu liều văcxin bại liệt mỗi năm, góp phần vào việc thanh toán hoàn toàn bệnh bại liệt tại Việt Nam vào những năm 2000. Ngoài ra, khỉ vàng cũng được sử dụng trong sản xuất vacxin chống bệnh bại liệt trẻ em, làm vật mẫu, đối tượng nghiên cứu khoa học. Đặc biệt, khỉ vàng được lựa chọn là đối tượng nghiên cứu của các đề tài khoa học cũng như kiểm định chất lượng, thử nghiệm tiền lâm sàng nhiều sản phẩm sinh học và văcxin.\n\n3. **Khỉ nâu**: Thông tin chỉ đề cập đến việc khỉ nâu là loài rất hung dữ và có một quần thể sinh sôi nảy nở trong các khu đầm lầy của Flo

# Baseline inference only using large model

In [ ]:
import os
import json
import requests
import pandas as pd
import re
import time
from dotenv import load_dotenv
from tqdm import tqdm

# 1. SETUP ENVIRONMENT & API
# ---------------------------------------------------------
load_dotenv()

LARGE_AUTH = os.getenv("LARGE_AUTH")
LARGE_TOKEN_KEY = os.getenv("LARGE_TOKEN_KEY")
LARGE_TOKEN_ID = os.getenv("LARGE_TOKEN_ID")

API_URL = "https://api.idg.vnpt.vn/data-service/v1/chat/completions/vnptai-hackathon-large"
HEADERS = {
    "Authorization": LARGE_AUTH,
    "Token-id": LARGE_TOKEN_ID,
    "Token-key": LARGE_TOKEN_KEY, 
    "Content-Type": "application/json",
}

# 2. LOAD DATA
# ---------------------------------------------------------
# Đường dẫn tương đối từ folder notebooks ra folder data
input_path = "../data/test/test.json" 
output_path = "../data/submiss/submission.csv"

# Check file tồn tại
if not os.path.exists(input_path):
    print(f"❌ Không tìm thấy file: {input_path}")
else:
    with open(input_path, "r", encoding="utf-8") as file:
        test_data = json.load(file)
    print(f"✅ Đã load {len(test_data)} câu hỏi từ {input_path}")

# 3. DEFINE INFERENCE FUNCTION
# ---------------------------------------------------------
def get_model_prediction(qid, question, choices):
    # Format choices thành chuỗi A. ... B. ...
    letters = ['A', 'B', 'C', 'D']
    choices_str = ""
    for idx, choice in enumerate(choices):
        if idx < 4:
            choices_str += f"{letters[idx]}. {choice}\n"
    
    # Prompt Engineering: Ép model suy luận và trả về định dạng <ANSWER>X</ANSWER> để dễ regex
    prompt = f"""
Bạn là một trợ lý AI thông minh giải quyết các câu hỏi trắc nghiệm.

NHIỆM VỤ:
1. Đọc kỹ câu hỏi và các lựa chọn.
2. Suy luận logic từng bước để tìm đáp án đúng nhất.
3. KẾT QUẢ CUỐI CÙNG bắt buộc phải đặt trong thẻ <ANSWER> theo định dạng: <ANSWER>Ký tự đáp án</ANSWER> (Ví dụ: <ANSWER>A</ANSWER>).

INPUT:
Câu hỏi: {question}
Các lựa chọn:
{choices_str}

Hãy suy nghĩ và đưa ra câu trả lời:
"""
    
    payload = {
        "model": "vnptai_hackathon_large",
        "messages": [
            {"role": "system", "content": "Bạn là chuyên gia giải đề thi trắc nghiệm chính xác."},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.1,  # Giảm nhiệt độ để model ổn định, bớt sáng tạo linh tinh
        "top_p": 0.9,
        "n": 1,
        "max_completion_tokens": 800, # Đủ cho suy luận CoT
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload, timeout=30)
        
        if response.status_code == 200:
            content = response.json()['choices'][0]['message']['content']
            
            # 1. Ưu tiên tìm trong thẻ <ANSWER>
            match = re.search(r"<ANSWER>\s*([A-D])\s*</ANSWER>", content, re.IGNORECASE)
            if match:
                return match.group(1).upper()
            
            # 2. Nếu không có thẻ, tìm ký tự A-D ở cuối cùng văn bản
            # Tìm pattern như "Đáp án: A" hoặc "Chọn A" hoặc chỉ đơn giản là "A" cuối câu
            fallback_match = re.findall(r"(?:đáp án|chọn|là)\s*[:]*\s*([A-D])", content, re.IGNORECASE)
            if fallback_match:
                return fallback_match[-1].upper()
            
            # 3. Fallback cuối cùng: Scan toàn bộ xem có A,B,C,D nào đứng riêng lẻ không
            simple_match = re.findall(r"\b([A-D])\b", content)
            if simple_match:
                return simple_match[-1] # Lấy cái cuối cùng model nhắc đến
                
            return "A" # Default nếu model nói linh tinh
            
        else:
            print(f"⚠️ Error {qid}: {response.status_code} - {response.text}")
            return "A" # Default error
            
    except Exception as e:
        print(f"❌ Exception {qid}: {str(e)}")
        return "A"

# 4. RUN INFERENCE LOOP (VERSION CÓ LOG CHI TIẾT)
# ---------------------------------------------------------
results = []
total_count = len(test_data)

print(f"🚀 Bắt đầu chạy Inference cho {total_count} câu hỏi...\n")

for i, item in enumerate(test_data):
    # Lấy thông tin
    qid = item.get('id') if 'id' in item else item.get('qid') 
    question = item['question']
    choices = item['choices']
    
    # In ra dòng thông báo TRƯỚC khi gọi API để biết đang làm câu nào
    # end="" giúp in tiếp trên cùng 1 dòng sau khi xong
    print(f"[{i+1}/{total_count}] Processing {qid}...", end=" ", flush=True)
    
    start_time = time.time()
    
    # Gọi hàm xử lý
    ans = get_model_prediction(qid, question, choices)
    
    # Tính thời gian chạy câu đó
    elapsed = time.time() - start_time
    
    # Lưu kết quả
    results.append({"qid": qid, "answer": ans})
    
    # In ra kết quả ngay lập tức
    print(f"✅ {elapsed:.1f}s -> Ans: {ans}")

# 5. SAVE SUBMISSION
# ---------------------------------------------------------
print("\n💾 Đang lưu file submission...")
df_sub = pd.DataFrame(results)

# Đảm bảo thư mục tồn tại
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df_sub.to_csv(output_path, index=False)

print(f"🎉 Hoàn tất! File saved tại: {output_path}")
print("5 dòng đầu tiên:")
print(df_sub.head())

✅ Đã load 370 câu hỏi từ ../data/test/test.json
🚀 Bắt đầu chạy Inference...


  3%|▎         | 12/370 [03:05<1:32:19, 15.47s/it]


KeyboardInterrupt: 